# TP 5. Ensemble Learning

> Wisdom of the crowd *(la sagesse de la foule)*




In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [15]:
df = pd.read_csv("wine_quality.csv", sep=",")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [16]:
# target
y = df.quality

# Features
X = df.drop(['quality','Id'], axis = 1).values

In [17]:
# Création des sous-ensembles d'entraînement et de test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Résultats obtenus par les différents modèles que nous avons vus

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [19]:
model_1 = LogisticRegression(max_iter=10000)
model_2 = KNeighborsClassifier(n_neighbors=2)
model_3 = DecisionTreeClassifier(max_depth=5)


for model in (model_1, model_2, model_3):
    model.fit(X_train, y_train)
    print(model.__class__.__name__, model.score(X_test, y_test))

LogisticRegression 0.6462882096069869
KNeighborsClassifier 0.5065502183406113
DecisionTreeClassifier 0.5764192139737991


## Voting Classifier

L'idée est de faire tourner plusieurs modèles en parallèle, puis de comparer leurs prédictions et de conserver celle qui est majoritaire.

### Vote hard


> Si 2 modèles prédisent une qualité de 5, et le 3ème modèle une probabilité de 6 alors la majorité l'emporte et la prédiction est de 5



In [20]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier([('Logistic', model_1),
                            ('KNN', model_2),
                            ('Tree', model_3)],
                          voting='hard')
model.fit(X_train, y_train)
print(model.__class__.__name__, model.score(X_test, y_test))

VotingClassifier 0.6026200873362445


### Vote soft


> Cette fois, on fait la moyenne des probabilités pour décider du résultat du vote. Par exemple :

*   model_1 prédit une qualité de 5 avec une probabilité de 0,51 et une qualité de 6 avec une probabilité de 0,49
*   model_2 prédit une qualité de 5 avec une probabilité de 0,55 et une qualité de 6 avec une probabilité de 0,45
*   model_3 prédit une qualité de 5 avec une probabilité de 0,35 et une qualité de 6 avec une probabilité de 0,65

alors la prédiction sera de (0,51+0,55+0,35)/3 = 0,47 pour la qualité de 5 et de (0,49+0,45+0,65)/3 = 0,53 pour la qualité de 6 => le modèle prévoit la qualité 6

In [21]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier([('Logistic', model_1),
                            ('KNN', model_2),
                            ('Tree', model_3)],
                          voting='soft')
model.fit(X_train, y_train)
print(model.__class__.__name__, model.score(X_test, y_test))

VotingClassifier 0.6462882096069869


## Bagging


> Cette fois-ci, c'est 1 seul type de modèle qui est choisi mais il est exécuté sur plusieurs sous-ensembles aléatoires du dataset, puis le résultat majoritaire est conservé.



### Bagging Classifier

In [22]:
from sklearn.ensemble import BaggingClassifier

In [23]:
model = BaggingClassifier(estimator=KNeighborsClassifier(),
                         n_estimators=100)

model.fit(X_train, y_train)
model.score(X_test, y_test)

0.5240174672489083

### Random Forest


> C'est du bagging, mais avec un arbre de décision comme modèle de base



In [24]:
model = RandomForestClassifier(n_estimators=100)

model.fit(X_train, y_train)
model.score(X_test, y_test)

0.6681222707423581

## Boosting

> Le Boosting consiste à enchaîner plusieurs modèles à la suite, chaque modèle essayant de corriger les faiblesses / erreurs des précédents. Il existe plusieurs algorithmes différents.



### AdaBoost

In [25]:
from sklearn.ensemble import AdaBoostClassifier

In [26]:
model = AdaBoostClassifier(n_estimators=100, algorithm='SAMME')
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.4847161572052402

### GradientBoost

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=100)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.6550218340611353

### Histogram Based GradientBoosting



In [28]:
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier(max_iter=100)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.6506550218340611

## Stacking

> Le stacking consiste à faire tourner différents modèles en parallèle, puis à utiliser un dernier modèle pour faire le choix de la prédiction à conserver.



In [29]:
from sklearn.ensemble import StackingClassifier

model_1 = LogisticRegression(max_iter=100000)
model_2 = KNeighborsClassifier(n_neighbors=2)
model_3 = DecisionTreeClassifier(max_depth=2)

final_model = LogisticRegression(max_iter=10000)

model = StackingClassifier(estimators=[('Logistic', model_1),
                            ('KNN', model_2),
                            ('Tree', model_3)],
                          final_estimator=final_model)
model.fit(X_train, y_train)
print(model.__class__.__name__, model.score(X_test, y_test))

StackingClassifier 0.6943231441048034
